In [1]:
# created on Dec 24, 2020
# modified on April 12, 2022
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

In [1]:
import tweepy, json, time
import pandas as pd
from google.colab import files
# Create data on to Google Drive
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.result = []
        self.f = file
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            # print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    row = {
                        'id': id,
                        'username': username,
                        'created_at': created_at,
                        'lng': lng,
                        'lat': lat,
                        'text': text
                    }
                    print(row)
                    self.result.append(row)
                else:
                    pass
        else:
            df = pd.DataFrame(self.result)
            df.to_csv(self.f, index=False)
            # download the csv to your local computer
            files.download(self.f)
            print("the csv has been downloaded to your local computer. The program has been completed successfully.")
            return False

In [ ]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = '/gdrive/My Drive/geotweets.csv'

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "me0cQe4Ox4w7zfWLnoEaYcjSI"
    consumer_secret = "LGdzzYT80XIHxoBKZt4HW4khpfVv1wUF16LjdbFlKoVkgBEaGf"
    access_token = "1516885989775405056-cSBebsdPWGSYutjXTTRJZdraS4vcmF"
    access_token_secret = "H7Mk52V8TAlE0E1TRkoF56PfA9kSomnBKXUbjRpbZ4OLY"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    '''
    LOCATIONS = [-124.7771694, 24.520833, -66.947028, 49.384472,  # Contiguous US
                 -164.639405, 58.806859, -144.152365, 71.76871,  # Alaska
                 -160.161542, 18.776344, -154.641396, 22.878623]  # Hawaii
    '''
    LOCATIONS = [-124.7771694, 24.520833, -66.947028, 49.384472]  # Contiguous US
    stream_listener = StreamListener(time_limit=300, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=LOCATIONS, languages=['en'], encoding="utf-8")
    stream.filter(track=['Three Body Problem'])

{'id': 1517268575425376260, 'username': 'Marathoncupcake', 'created_at': 'Thu Apr 21 22:26:32 +0000 2022', 'lng': -75.117998, 'lat': 40.0048655, 'text': 'MOST OF US GOING THROUGH THINGS NOBODY KNOW ABOUT. PRAY 🙏🏿❤️ FOR EVERYBODY'}
{'id': 1517268575119155200, 'username': 'carigi12', 'created_at': 'Thu Apr 21 22:26:32 +0000 2022', 'lng': -83.895515, 'lat': 33.977698000000004, 'text': '@la_nachap This attorney is flambasting his ass! https://t.co/3BN8Lk6isn'}
{'id': 1517268575983124481, 'username': 'KQuiltMDA', 'created_at': 'Thu Apr 21 22:26:32 +0000 2022', 'lng': -87.848928, 'lat': 41.499327, 'text': "@AmyAThatcher I've never watched it and now, I never will."}
{'id': 1517268576700301312, 'username': 'ThankYouTaceGod', 'created_at': 'Thu Apr 21 22:26:32 +0000 2022', 'lng': -118.4119065, 'lat': 34.0207895, 'text': '@bonustr30205373 @okcardinalsfan @ThomasHoops87 @joeylinn_ Bucks &amp; suns had two of the luckiest playoff runs I’ve e… https://t.co/HSs4LNO3Vb'}
{'id': 1517268577673519106, 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.
